In [ ]:
import os
import openai
from DataAnalyzer import DataAnalyzer

from dotenv import load_dotenv
load_dotenv(".env")

### SET VARIABLES

In [ ]:
aoai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
aoai_api_key = os.environ["AZURE_OPENAI_API_KEY"]
api_version = os.environ["AZURE_OPENAI_API_VERSION"]

aoai_client = openai.AzureOpenAI( 
    azure_endpoint=aoai_endpoint,
    api_key=aoai_api_key,
    api_version= api_version
)
    
chat_model: str = os.environ["AZURE_OPENAI_CHAT_MODEL"]

In [ ]:
analyzer = DataAnalyzer(aoai_client, chat_model)
tools = DataAnalyzer.TOOLS

### SET MESSAGES

In [ ]:
file_name = "./data/anomaly_score_02.csv"

with open("./user_promp_template.txt", "r", encoding = 'utf-8') as file:
    developer_message = file.read()
    developer_message = developer_message.format(file_name)
with open(file_name, "r", encoding = 'utf-8-sig') as csvfile:
    developer_data = csvfile.read()

user_context = """
    The data indicates anomaly in audio signal as time series and should be all zero for normal state. 
    The left-most column is a time column. Each column shows different types of anomaly. 
    The column \"test-data\" belongs to on of the type. Which type does it fall into?  
    """

messages = []
messages.append({"role": "developer","content": developer_message})
messages.append({"role": "developer","content": developer_data})
messages.append({"role": "user", "content": user_context})

<h5> WITH FUNCTION CALLING </h5>

In [ ]:
response_message, messages = analyzer.call_o1_model(messages, tools=tools, tool_choice='auto')
print(response_message.content)

### Follow up question

In [ ]:
followup_question = "他の可能性を教えてください。符号は絶対値も考慮してください。"
messages.append({"role": "user", "content": followup_question})

response_message, messages = analyzer.call_o1_model(messages)
print(response_message.content)